Lable encoding

если порядок совпадает с ожидаемым, то есть по алфавиту буквы идут и подходит, что будет от 0 до n  
или нам не важно какая буква будет с какой цифорой

In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

l = ['a', 'c', 'b', 'd', 'e']

le.fit_transform(l)

array([0, 2, 1, 3, 4], dtype=int64)

если нам нужно правильный порядок сделать, то используем label_binarize  
label_binarize делает one hot encoding, и если нам нужно от 0 до n, то используем argmax

In [11]:
from sklearn.preprocessing import label_binarize

l = ['bad', 'excellent', 'medium', 'very good', 'very bad']

label_binarize(l, classes=['excellent', 'very good', 'medium', 'bad', 'very bad']).argmax(axis=1)

array([3, 0, 2, 1, 4], dtype=int64)

In [12]:
from sklearn.preprocessing import label_binarize

l = ['bad', 'excellent', 'medium', 'very good', 'very bad']

label_binarize(l, classes=['excellent', 'very good', 'medium', 'bad', 'very bad'])

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1]])